# Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

#!pip install investpy
import investpy as inv
from time import sleep
import time

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def converter_valor(valor):
    if valor.endswith('B'):
        return float(valor.replace('B', '').replace(' ', '')) * 1_000
    elif valor.endswith('M'):
        return float(valor.replace('M', '').replace(' ', '')) * 1
    elif valor.endswith('K'):
        return float(valor.replace('K', '').replace(' ', '')) * 0.001
    elif valor == 'NA':
        return float('nan')

# Coletando os dados

In [5]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

# Acertando

In [7]:
refazer_comeco = False

In [8]:
if refazer_comeco:
    servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
    # Iniciar o navegador com o serviço
    navegador = webdriver.Chrome(service=servico)
    #setor_financeiro = ['BBAS3','BBDC3']
    setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11'}
    lista_dataframes = []
    iteracao=0
    
    for acao in codigos_ibovespa:
        if acao in setor_financeiro: 
            print(f'Execucao da {acao} do setor financeiro.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                
                # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
    
                    # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                preco_lucro.append(lista_preco_relativo[1])
                preco_vpa.append(lista_preco_relativo[3])
                preco_receita_liquida.append(lista_preco_relativo[5])
                preco_fco.append(lista_preco_relativo[7])
                preco_fcf.append(np.nan)
                preco_ativo_total.append(lista_preco_relativo[9])
                preco_ebit.append(np.nan)
                preco_capital_giro.append(np.nan)
                preco_ncav.append(np.nan)
                ev_ebit.append(np.nan)
                ev_ebitda.append(np.nan)
                ev_receita_liquida.append(np.nan)
                ev_fco.append(np.nan)
                ev_fcf.append(np.nan)
                ev_atito_total.append(np.nan)
                market_cap_empresa.append(lista_preco_relativo[11])
                enterprise_value.append(np.nan)
                dividend_yield.append(lista_preco_relativo[-1])
            
        else:
            
            print(f'Execucao da {acao} do setor financeiro.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            # Encontrar a tabela com o id
            
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                 # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
    
                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                preco_lucro.append(lista_preco_relativo[1])
                preco_vpa.append(lista_preco_relativo[3])
                preco_receita_liquida.append(lista_preco_relativo[5])
                preco_fco.append(lista_preco_relativo[7])
                preco_fcf.append(lista_preco_relativo[9])
                preco_ativo_total.append(lista_preco_relativo[11])
                preco_ebit.append(lista_preco_relativo[13])
                preco_capital_giro.append(lista_preco_relativo[15])
                preco_ncav.append(lista_preco_relativo[17])
                ev_ebit.append(lista_preco_relativo[19])
                ev_ebitda.append(lista_preco_relativo[21])
                ev_receita_liquida.append(lista_preco_relativo[23])
                ev_fco.append(lista_preco_relativo[25])
                ev_fcf.append(lista_preco_relativo[27])
                ev_atito_total.append(lista_preco_relativo[29])
                market_cap_empresa.append(lista_preco_relativo[31])
                enterprise_value.append(lista_preco_relativo[33])
                dividend_yield.append(lista_preco_relativo[-1])
            
        listas_dados = {
            'data':datas,
            'preco_lucro': preco_lucro,
            'preco_vpa': preco_vpa,
            'preco_receita_liquida': preco_receita_liquida,
            'preco_fco': preco_fco,
            'preco_fcf': preco_fcf,
            'preco_ativo_total': preco_ativo_total,
            'preco_ebit':preco_ebit,
            'preco_capital_giro': preco_capital_giro,
            'preco_ncav': preco_ncav,
            'ev_ebit': ev_ebit,
            'ev_ebitda': ev_ebitda,
            'ev_receita_liquida': ev_receita_liquida,
            'ev_fco': ev_fco,
            'ev_fcf': ev_fcf,
            'ev_atito_total': ev_atito_total,
            'market_cap_empresa': market_cap_empresa,
            'enterprise_value': enterprise_value,
            'dividend_yield': dividend_yield
        }
        df_preco_relativo = pd.DataFrame(listas_dados)
        
        if acao in setor_financeiro:
            colunas_selecionadas = df_preco_relativo.columns[(df_preco_relativo.columns != 'data') & 
                                    (df_preco_relativo.columns != 'preco_fcf') &
                                    (df_preco_relativo.columns != 'preco_ebit') &   
                                    (df_preco_relativo.columns != 'preco_capital_giro') & 
                                    (df_preco_relativo.columns != 'preco_ncav') &
                                    (df_preco_relativo.columns != 'ev_ebit') &  
                                    (df_preco_relativo.columns != 'ev_ebitda')& 
                                    (df_preco_relativo.columns != 'ev_ebitda')&
                                    (df_preco_relativo.columns != 'ev_receita_liquida')&
                                    (df_preco_relativo.columns != 'ev_fco')&
                                    (df_preco_relativo.columns != 'ev_fcf')&
                                    (df_preco_relativo.columns != 'ev_atito_total')&
                                    (df_preco_relativo.columns != 'enterprise_value')]   
            
            for col in colunas_selecionadas:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)   
        else:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns != 'data') & (df_preco_relativo.columns != 'lucro_por_acao')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)
                
        df_preco_relativo['tic'] = acao
        
        lista_dataframes.append(df_preco_relativo)
        
        iteracao = iteracao+1
        
        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
        
        end_time = time.time()
        
        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time
    
        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60
    
        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

In [17]:
data_frame_feito = pd.read_csv('precos_relativos_trimestrais_2.csv').iloc[:,1:]

In [19]:
data_frame_feito.tic.unique()

array(['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3',
       'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CCRO3', 'CIEL3',
       'CMIG4', 'COGN3', 'CPFE3', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3',
       'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENGI11',
       'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
       'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
       'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3',
       'PETR4', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RENT3', 'SANB11',
       'SBSP3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3'], dtype=object)

In [20]:
servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
# Iniciar o navegador com o serviço
navegador = webdriver.Chrome(service=servico)
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11','IRBR3'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        if acao in setor_financeiro: 
            print(f'Execucao da {acao} do setor financeiro.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                
                # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
    
                    # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                preco_lucro.append(lista_preco_relativo[1])
                preco_vpa.append(lista_preco_relativo[3])
                preco_receita_liquida.append(lista_preco_relativo[5])
                preco_fco.append(lista_preco_relativo[7])
                preco_fcf.append(np.nan)
                preco_ativo_total.append(lista_preco_relativo[9])
                preco_ebit.append(np.nan)
                preco_capital_giro.append(np.nan)
                preco_ncav.append(np.nan)
                ev_ebit.append(np.nan)
                ev_ebitda.append(np.nan)
                ev_receita_liquida.append(np.nan)
                ev_fco.append(np.nan)
                ev_fcf.append(np.nan)
                ev_atito_total.append(np.nan)
                market_cap_empresa.append(lista_preco_relativo[11])
                enterprise_value.append(np.nan)
                dividend_yield.append(lista_preco_relativo[-1])
            
        else:
            
            print(f'Execucao da {acao}.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            # Encontrar a tabela com o id
            
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                 # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
    
                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                preco_lucro.append(lista_preco_relativo[1])
                preco_vpa.append(lista_preco_relativo[3])
                preco_receita_liquida.append(lista_preco_relativo[5])
                preco_fco.append(lista_preco_relativo[7])
                preco_fcf.append(lista_preco_relativo[9])
                preco_ativo_total.append(lista_preco_relativo[11])
                preco_ebit.append(lista_preco_relativo[13])
                preco_capital_giro.append(lista_preco_relativo[15])
                preco_ncav.append(lista_preco_relativo[17])
                ev_ebit.append(lista_preco_relativo[19])
                ev_ebitda.append(lista_preco_relativo[21])
                ev_receita_liquida.append(lista_preco_relativo[23])
                ev_fco.append(lista_preco_relativo[25])
                ev_fcf.append(lista_preco_relativo[27])
                ev_atito_total.append(lista_preco_relativo[29])
                market_cap_empresa.append(lista_preco_relativo[31])
                enterprise_value.append(lista_preco_relativo[33])
                dividend_yield.append(lista_preco_relativo[-1])
            
        listas_dados = {
            'data':datas,
            'preco_lucro': preco_lucro,
            'preco_vpa': preco_vpa,
            'preco_receita_liquida': preco_receita_liquida,
            'preco_fco': preco_fco,
            'preco_fcf': preco_fcf,
            'preco_ativo_total': preco_ativo_total,
            'preco_ebit':preco_ebit,
            'preco_capital_giro': preco_capital_giro,
            'preco_ncav': preco_ncav,
            'ev_ebit': ev_ebit,
            'ev_ebitda': ev_ebitda,
            'ev_receita_liquida': ev_receita_liquida,
            'ev_fco': ev_fco,
            'ev_fcf': ev_fcf,
            'ev_atito_total': ev_atito_total,
            'market_cap_empresa': market_cap_empresa,
            'enterprise_value': enterprise_value,
            'dividend_yield': dividend_yield
        }
        df_preco_relativo = pd.DataFrame(listas_dados)
        
        if acao in setor_financeiro:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)   
        else:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa') | (df_preco_relativo.columns == 'enterprise_value')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)
                
        df_preco_relativo['tic'] = acao
        
        lista_dataframes.append(df_preco_relativo)
        
        iteracao = iteracao+1
        
        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
        
        end_time = time.time()
        
        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time
    
        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60
    
        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Essa ABEV3 ja foi
Essa AZUL4 ja foi
Essa B3SA3 ja foi
Essa BBAS3 ja foi
Essa BBDC3 ja foi
Essa BBDC4 ja foi
Essa BBSE3 ja foi
Essa BEEF3 ja foi
Essa BPAC11 ja foi
Essa BRAP4 ja foi
Essa BRFS3 ja foi
Essa BRKM5 ja foi
Essa CCRO3 ja foi
Essa CIEL3 ja foi
Essa CMIG4 ja foi
Essa COGN3 ja foi
Essa CPFE3 ja foi
Essa CRFB3 ja foi
Essa CSAN3 ja foi
Essa CSNA3 ja foi
Essa CVCB3 ja foi
Essa CYRE3 ja foi
Essa ECOR3 ja foi
Essa EGIE3 ja foi
Essa ELET3 ja foi
Essa ELET6 ja foi
Essa EMBR3 ja foi
Essa ENGI11 ja foi
Essa EQTL3 ja foi
Essa EZTC3 ja foi
Essa FLRY3 ja foi
Essa GGBR4 ja foi
Essa GOAU4 ja foi
Essa GOLL4 ja foi
Essa HAPV3 ja foi
Essa HYPE3 ja foi
Essa IRBR3 ja foi
Essa ITSA4 ja foi
Essa ITUB4 ja foi
Essa JBSS3 ja foi
Essa KLBN11 ja foi
Essa LREN3 ja foi
Essa MGLU3 ja foi
Essa MRFG3 ja foi
Essa MRVE3 ja foi
Essa MULT3 ja foi
Essa NTCO3 ja foi
Essa PCAR3 ja foi
Essa PETR3 ja foi
Essa PETR4 ja foi
Essa PRIO3 ja foi
Essa QUAL3 ja foi
Essa RADL3 ja foi
Essa RAIL3 ja foi
Essa RENT3 ja foi
Essa SA

In [21]:
pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais_3.csv')

In [43]:
servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
# Iniciar o navegador com o serviço
navegador = webdriver.Chrome(service=servico)
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
lista_dataframes = []
iteracao=0
for acao in codigos_ibovespa:
    if acao in setor_financeiro:
        print(f'Execucao da {acao} do setor financeiro.')
        #Comeca o tempo
        start_time = time.time()
        #Indica a pagina web
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')

        # Extrair o texto de todas as datas //*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select
        #elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
        elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select')
        datas = [elemento.text for elemento in elementos_data]

        if len(datas) == 1:
            string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
            datas = string_de_datas.split('\n')
        preco_lucro = []
        preco_vpa  = []
        preco_receita_liquida = []
        preco_fco = []
        preco_fcf = []
        preco_ativo_total = []
        preco_ebit = []
        preco_capital_giro = []
        preco_ncav = []
        ev_ebit = []
        ev_ebitda = []
        ev_receita_liquida=[]
        ev_fco = []
        ev_fcf =[]
        ev_atito_total =[]
        market_cap_empresa = []
        enterprise_value =[]
        dividend_yield = []

        for i in datas:

            elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
            # Crie um objeto Select
            select = Select(elemento_select)
            # Altere o período selecionando por texto visível
            select.select_by_visible_text(i)
            servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')

            # Pegar as linhas da tabela
            linhas = tabela.find_elements(By.TAG_NAME, 'tr')

            lista_preco_relativo = []
            for linha in linhas:
                # Pegar as células de cada linha
                celulas = linha.find_elements(By.TAG_NAME, 'td')
                for celula in celulas:
                    lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                    
            preco_lucro.append(lista_preco_relativo[1])
            preco_vpa.append(lista_preco_relativo[3])
            preco_receita_liquida.append(lista_preco_relativo[5])
            preco_fco.append(lista_preco_relativo[7])
            preco_fcf.append(np.nan)
            preco_ativo_total.append(lista_preco_relativo[9])
            preco_ebit.append(np.nan)
            preco_capital_giro.append(np.nan)
            preco_ncav.append(np.nan)
            ev_ebit.append(np.nan)
            ev_ebitda.append(np.nan)
            ev_receita_liquida.append(np.nan)
            ev_fco.append(np.nan)
            ev_fcf.append(np.nan)
            ev_atito_total.append(np.nan)
            market_cap_empresa.append(lista_preco_relativo[11])
            enterprise_value.append(np.nan)
            dividend_yield.append(lista_preco_relativo[-1])
    
    else:
        print(f'Execucao da {acao}.')
        #Comeca o tempo
        start_time = time.time()
        #Indica a pagina web
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')

        # Extrair o texto de todas as datas //*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select
        #elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
        elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select')
        datas = [elemento.text for elemento in elementos_data]

        if len(datas) == 1:
            string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
            datas = string_de_datas.split('\n')
        preco_lucro = []
        preco_vpa  = []
        preco_receita_liquida = []
        preco_fco = []
        preco_fcf = []
        preco_ativo_total = []
        preco_ebit = []
        preco_capital_giro = []
        preco_ncav = []
        ev_ebit = []
        ev_ebitda = []
        ev_receita_liquida=[]
        ev_fco = []
        ev_fcf =[]
        ev_atito_total =[]
        market_cap_empresa = []
        enterprise_value =[]
        dividend_yield = []

        for i in datas:

            elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
            # Crie um objeto Select
            select = Select(elemento_select)
            # Altere o período selecionando por texto visível
            select.select_by_visible_text(i)
            servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')

            # Pegar as linhas da tabela
            linhas = tabela.find_elements(By.TAG_NAME, 'tr')

            lista_preco_relativo = []
            for linha in linhas:
                # Pegar as células de cada linha
                celulas = linha.find_elements(By.TAG_NAME, 'td')
                for celula in celulas:
                    lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

            preco_lucro.append(lista_preco_relativo[1])
            preco_vpa.append(lista_preco_relativo[3])
            preco_receita_liquida.append(lista_preco_relativo[5])
            preco_fco.append(lista_preco_relativo[7])
            preco_fcf.append(lista_preco_relativo[9])
            preco_ativo_total.append(lista_preco_relativo[11])
            preco_ebit.append(lista_preco_relativo[13])
            preco_capital_giro.append(lista_preco_relativo[15])
            preco_ncav.append(lista_preco_relativo[17])
            ev_ebit.append(lista_preco_relativo[19])
            ev_ebitda.append(lista_preco_relativo[21])
            ev_receita_liquida.append(lista_preco_relativo[23])
            ev_fco.append(lista_preco_relativo[25])
            ev_fcf.append(lista_preco_relativo[27])
            ev_atito_total.append(lista_preco_relativo[29])
            market_cap_empresa.append(lista_preco_relativo[31])
            enterprise_value.append(lista_preco_relativo[33])
            dividend_yield.append(lista_preco_relativo[-1])

    listas_dados = {
            'data':datas,
            'preco_lucro': preco_lucro,
            'preco_vpa': preco_vpa,
            'preco_receita_liquida': preco_receita_liquida,
            'preco_fco': preco_fco,
            'preco_fcf': preco_fcf,
            'preco_ativo_total': preco_ativo_total,
            'preco_ebit':preco_ebit,
            'preco_capital_giro': preco_capital_giro,
            'preco_ncav': preco_ncav,
            'ev_ebit': ev_ebit,
            'ev_ebitda': ev_ebitda,
            'ev_receita_liquida': ev_receita_liquida,
            'ev_fco': ev_fco,
            'ev_fcf': ev_fcf,
            'ev_atito_total': ev_atito_total,
            'market_cap_empresa': market_cap_empresa,
            'enterprise_value': enterprise_value,
            'dividend_yield': dividend_yield
        }
    df_preco_relativo = pd.DataFrame(listas_dados)
    
    if acao in setor_financeiro:
        for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa')]:
            df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)   
    else:
        for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa') | (df_preco_relativo.columns == 'enterprise_value')]:
            df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)
            
    df_preco_relativo['tic'] = acao
    
    lista_dataframes.append(df_preco_relativo[1:])
    
    iteracao = iteracao+1
    
    if iteracao % 10 == 0:
        pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
    
    end_time = time.time()
    
    # Calcula o tempo decorrido em segundos
    execution_time_seconds = end_time - start_time

    # Calcula o tempo decorrido em minutos
    execution_time_minutes = execution_time_seconds / 60

    # Imprime o tempo de execução em minutos
    print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Execucao da ABEV3.
Tempo de execução da ação ABEV3: 1.87 minutos
Execucao da AZUL4.
Tempo de execução da ação AZUL4: 1.19 minutos
Execucao da B3SA3.
Tempo de execução da ação B3SA3: 2.53 minutos
Execucao da BBAS3 do setor financeiro.
Tempo de execução da ação BBAS3: 3.30 minutos
Execucao da BBDC3 do setor financeiro.
Tempo de execução da ação BBDC3: 3.36 minutos
Execucao da BBDC4 do setor financeiro.
Tempo de execução da ação BBDC4: 3.81 minutos
Execucao da BBSE3 do setor financeiro.
Tempo de execução da ação BBSE3: 2.14 minutos
Execucao da BEEF3.
Tempo de execução da ação BEEF3: 3.54 minutos
Execucao da BPAC11 do setor financeiro.
Tempo de execução da ação BPAC11: 2.37 minutos
Execucao da BRAP4.
Tempo de execução da ação BRAP4: 4.64 minutos
Execucao da BRFS3.
Tempo de execução da ação BRFS3: 4.57 minutos
Execucao da BRKM5.
Tempo de execução da ação BRKM5: 4.46 minutos
Execucao da CCRO3.
Tempo de execução da ação CCRO3: 4.45 minutos
Execucao da CIEL3.
Tempo de execução da ação CIEL3: 2

IndexError: list index out of range

In [35]:
servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
# Iniciar o navegador com o serviço
navegador = webdriver.Chrome(service=servico)
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
lista_dataframes = []
iteracao=0
for acao in codigos_ibovespa:
    print(f'Execucao da {acao} do setor financeiro.')
    #Comeca o tempo
    start_time = time.time()
    #Indica a pagina web
    servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Iniciar o navegador com o serviço
    navegador = webdriver.Chrome(service=servico)
    navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')

    # Extrair o texto de todas as datas //*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select
    #elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
    elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select')
    datas = [elemento.text for elemento in elementos_data]

    if len(datas) == 1:
        string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
        datas = string_de_datas.split('\n')
    preco_lucro = []
    preco_vpa  = []
    preco_receita_liquida = []
    preco_fco = []
    preco_fcf = []
    preco_ativo_total = []
    preco_ebit = []
    preco_capital_giro = []
    preco_ncav = []
    ev_ebit = []
    ev_ebitda = []
    ev_receita_liquida=[]
    ev_fco = []
    ev_fcf =[]
    ev_atito_total =[]
    market_cap_empresa = []
    enterprise_value =[]
    dividend_yield = []

    for i in datas:

        elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
        # Crie um objeto Select
        select = Select(elemento_select)
        # Altere o período selecionando por texto visível
        select.select_by_visible_text(i)
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

        # Encontrar a tabela com o id
        tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')

        # Pegar as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')

        lista_preco_relativo = []
        for linha in linhas:
            # Pegar as células de cada linha
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            for celula in celulas:
                lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

        preco_lucro.append(lista_preco_relativo[1])
        preco_vpa.append(lista_preco_relativo[3])
        preco_receita_liquida.append(lista_preco_relativo[5])
        preco_fco.append(lista_preco_relativo[7])
        preco_fcf.append(lista_preco_relativo[9])
        preco_ativo_total.append(lista_preco_relativo[11])
        preco_ebit.append(lista_preco_relativo[13])
        preco_capital_giro.append(lista_preco_relativo[15])
        preco_ncav.append(lista_preco_relativo[17])
        ev_ebit.append(lista_preco_relativo[19])
        ev_ebitda.append(lista_preco_relativo[21])
        ev_receita_liquida.append(lista_preco_relativo[23])
        ev_fco.append(lista_preco_relativo[25])
        ev_fcf.append(lista_preco_relativo[27])
        ev_atito_total.append(lista_preco_relativo[29])
        market_cap_empresa.append(lista_preco_relativo[31])
        enterprise_value.append(lista_preco_relativo[33])
        dividend_yield.append(lista_preco_relativo[-1])

    listas_dados = {
            'data':datas,
            'preco_lucro': preco_lucro,
            'preco_vpa': preco_vpa,
            'preco_receita_liquida': preco_receita_liquida,
            'preco_fco': preco_fco,
            'preco_fcf': preco_fcf,
            'preco_ativo_total': preco_ativo_total,
            'preco_ebit':preco_ebit,
            'preco_capital_giro': preco_capital_giro,
            'preco_ncav': preco_ncav,
            'ev_ebit': ev_ebit,
            'ev_ebitda': ev_ebitda,
            'ev_receita_liquida': ev_receita_liquida,
            'ev_fco': ev_fco,
            'ev_fcf': ev_fcf,
            'ev_atito_total': ev_atito_total,
            'market_cap_empresa': market_cap_empresa,
            'enterprise_value': enterprise_value,
            'dividend_yield': dividend_yield
        }
    df_preco_relativo = pd.DataFrame(listas_dados)
    
    lista_dataframes.append(df_preco_relativo[1:])
    
    pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
    
    end_time = time.time()
    
    # Calcula o tempo decorrido em segundos
    execution_time_seconds = end_time - start_time

    # Calcula o tempo decorrido em minutos
    execution_time_minutes = execution_time_seconds / 60

    # Imprime o tempo de execução em minutos
    print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Execucao da ABEV3 do setor financeiro.
Tempo de execução da ação ABEV3: 3.53 minutos
Execucao da AZUL4 do setor financeiro.
Tempo de execução da ação AZUL4: 2.07 minutos
Execucao da B3SA3 do setor financeiro.
Tempo de execução da ação B3SA3: 3.52 minutos
Execucao da BBAS3 do setor financeiro.


IndexError: list index out of range

In [1]:
refazer_comeco = False